In [ ]:
import os
import os.path as path
import sys
sys.path.append("D:\\ASGaze")

import cv2
import numpy as np
import pandas as pd
import configparser
from glob import glob
from sklearn.metrics import mean_squared_error

import import_ipynb
from gaze_ray_estimator.cone_model import Cone, Ellipse
from mapping.mapping_principle import Line3, screen2cam_mm, cam2screen_mm

config = configparser.ConfigParser()

In [ ]:
def get_cone(ellipse_files_l,ellipse_files_r,crop_info,camera_matrix_inv,iris_radius):
    
    config.read(ellipse_files_l)
    center_l = (
        float(config.get('iris','center_x'))+crop_info[0],
        float(config.get('iris','center_y'))+crop_info[1]
    )
    long_l,short_l = (float(config.get('iris','long_radius'))*2,float(config.get('iris','short_radius'))*2)
    radius_l = [long_l,short_l]
    angle_l = float(config.get('iris','rad_phi'))
    rotated_rect_l = (center_l, radius_l, angle_l)
    ellipse_l = Ellipse(rect=rotated_rect_l) # Base plane of cone model
    
    r_l,c_l = ellipse_l.get_points()
    limbus_l = np.array([r_l,c_l]).T
    limbus_l = np.hstack([limbus_l, np.ones(shape=(len(r_l),1))]) # Image coordinate system
    limbus_l = limbus_l.dot(camera_matrix_inv.T) # Convert to camera coordinate system

    config.read(ellipse_files_r)
    center_r = (
        float(config.get('iris','center_x'))+crop_info[4],
        float(config.get('iris','center_y'))+crop_info[5]
    )
    long_r,short_r = (float(config.get('iris','long_radius'))*2,float(config.get('iris','short_radius'))*2)
    radius_r = [long_r,short_r]
    angle_r = float(config.get('iris','rad_phi'))
    rotated_rect_r = (center_r, radius_r, angle_r)
    ellipse_r = Ellipse(rect=rotated_rect_r)
    
    r_r,c_r = ellipse_r.get_points()
    limbus_r = np.array([r_r,c_r]).T
    limbus_r = np.hstack([limbus_r, np.ones(shape=(len(r_r),1))])
    limbus_r = limbus_r.dot(camera_matrix_inv.T)
    
    ellipse_3d_l = Ellipse(p=limbus_l[:,:2].astype(np.float32))# Camera coordinate system
    ellipse_3d_r = Ellipse(p=limbus_r[:,:2].astype(np.float32))
    
    vertex = [0,0,-1]
    cone_l = Cone(v=vertex, base=ellipse_3d_l, radius=iris_radius)# Camera coordinate system
    cone_r = Cone(v=vertex, base=ellipse_3d_r, radius=iris_radius)
    
    gaze_l = cone_l.norm # Gaze direction (camera coordinate system)
    gaze_origin_l = np.append(cone_l.get_position(0),cone_l.get_position(1)).reshape(2,4) # Gaze origin (camera coordinate system)
    
    gaze_r = cone_r.norm
    gaze_origin_r = np.append(cone_r.get_position(0),cone_r.get_position(1)).reshape(2,4)
    
    return rotated_rect_l,rotated_rect_r,gaze_l,gaze_r,gaze_origin_l,gaze_origin_r

### Gaze Tracker

In [ ]:
def tracker(p_model,frame_files,ellipse_files_l,ellipse_files_r,crop_info,gaze_label_mm,camera_matrix_inv,iris_radius,true_seq,offsets,window_size,video=False):
    screen_l_list = []
    screen_r_list = []
    screen_list = []
        
    x_error = []
    y_error = []
    
    if video:
        rotated_rect_l_list = []
        rotated_rect_r_list = []
        gaze_origin_l_list = []
        gaze_origin_r_list = []

    for i in range(0,len(frame_files)):
        rotated_rect_l,rotated_rect_r,gaze_l,gaze_r,gaze_origin_l,gaze_origin_r = get_cone(ellipse_files_l[i],ellipse_files_r[i],crop_info[i],camera_matrix_inv,iris_radius)
        
        gaze_l = gaze_l[true_seq] 
        gaze_r = gaze_r[true_seq]

        gaze_origin_l = gaze_origin_l[true_seq]
        gaze_origin_r = gaze_origin_r[true_seq]
        
        if video:
            rotated_rect_l_list.append(rotated_rect_l)
            rotated_rect_r_list.append(rotated_rect_r)
            gaze_origin_l_list.append(gaze_origin_l)
            gaze_origin_r_list.append(gaze_origin_r)

        gaze_line_l = Line3(p=gaze_origin_l[:3], normal=gaze_l)
        PoG_l = gaze_line_l.get_pos() # Camera coordinate system
        screen_l = cam2screen_mm(PoG_l,offsets[0],offsets[1])# Tracking area coordinate system
        screen_l_list.append(screen_l)

        gaze_line_r = Line3(p=gaze_origin_r[:3], normal=gaze_r)
        PoG_r = gaze_line_r.get_pos() # Camera coordinate system
        screen_r = cam2screen_mm(PoG_r,offsets[0],offsets[1])# Tracking area coordinate system
        screen_r_list.append(screen_r)

        PoG = 0.5*(PoG_l+PoG_r)
        screen = 0.5*(screen_l+screen_r)
        screen_list.append(screen)
        
        x_error.append(abs(screen[0] - gaze_label_mm[i][0]))
        y_error.append(abs(screen[1] - gaze_label_mm[i][1]))

    print("Gaze Ray Estimator Done")

    #-----------error-----------  
    x_error_best = min(x_error)
    x_error_best_idx = np.argmin(x_error)
    x_error_worst = max(x_error)
    x_error_worst_idx = np.argmax(x_error)

    y_error_best = min(y_error)
    y_error_best_idx = np.argmin(y_error)
    y_error_worst = max(y_error)
    y_error_worst_idx = np.argmax(y_error)
    
    # If True returns MSE value, if False returns RMSE value.
    rms = mean_squared_error(np.array(gaze_label_mm), np.array(screen_list), squared=False)
    print("-------------Error Calculation Done----------\n")
    print("x_axis_best:{a} mm, index:{b}\n".format(a=x_error_best, b=x_error_best_idx))
    print("x_axis_worst:{a} mm, index:{b}\n".format(a=x_error_worst, b=x_error_worst_idx))
    print("y_axis_best:{a} mm, index:{b}\n".format(a=y_error_best, b=y_error_best_idx))
    print("y_axis_worst:{a} mm, index:{b}\n".format(a=y_error_worst, b=y_error_worst_idx))
    print("overall error:{a} mm\n".format(a=rms))
    
    def smooth(a,n,mode="same"):
        return(np.convolve(a, np.ones((n,))/n, mode=mode))
 
    screen_l_np = np.array(screen_l_list)
    screen_r_np = np.array(screen_r_list)
    screen_np = np.array(screen_list)
    
    screen_left_refine_x = smooth(screen_l_np[:,0],window_size)
    screen_left_refine_y = smooth(screen_l_np[:,1],window_size)
    screen_left_refine = np.vstack((screen_left_refine_x,screen_left_refine_y)).T

    screen_right_refine_x = smooth(screen_r_np[:,0],window_size)
    screen_right_refine_y = smooth(screen_r_np[:,1],window_size)
    screen_right_refine = np.vstack((screen_right_refine_x,screen_right_refine_y)).T

    screen_refine = 0.5*(screen_left_refine+screen_right_refine)
    
    rms_refine = mean_squared_error(np.array(gaze_label_mm), screen_refine, squared=False)
    print("overall error after moving average filter:{a} mm\n".format(a=rms_refine))
    
    return rotated_rect_l_list,rotated_rect_r_list,gaze_origin_l_list,gaze_origin_r_list,screen_left_refine,screen_right_refine

In [ ]:
def load_files(p_root,test_name,model_id):
    p_user = path.join(p_root,test_name)
    p_raw = path.join(p_user,"frame")
    p_crop = path.join(p_user,"crop")

    p_model = path.join(p_user,model_id)
    p_gen_l = path.join(p_model,"InferResults","ellipse_params_left")
    p_gen_r = path.join(p_model,"InferResults","ellipse_params_right") # Load obtained ellipse parameters
    
    cam_path = "D:\\ASGaze\\Database\\camera_matrix\\calib.npz"
    camera_matrix,crop_info = np.load(cam_path),np.load(path.join(p_crop,"crop.npy")) # Load crop region info and camera matrix
    camera_matrix_inv = np.linalg.inv(camera_matrix['mtx'])
    
    gaze_label_mm = np.loadtxt(path.join(p_user,'GroundTruth.txt'))# Load ground truth of gaze point

    frame_files = glob(path.join(p_raw,"*.png"))
    ellipse_files_l = glob(path.join(p_gen_l,"*.ini"))
    ellipse_files_r = glob(path.join(p_gen_r,"*.ini"))
    
    return p_model,frame_files,ellipse_files_l,ellipse_files_r,crop_info,gaze_label_mm,camera_matrix_inv